### visulaizing the  corine images

### The corine land cover classes

In [1]:
clcc ={111:'cont. urban fabric',
 112:'disc urban fabric',
 121:'industrial or commercial units',
 122:'road and rail',
 123:'port areas',
 124:'airports',
 131:'mineral extraction sites',
 132:'dump sites',
 133:'construction sites',
 141:'green urban areas',
 142:'sport and leasure',
 211:'non irregated arable land',
 212:'permenant irregated land',
 213:'rice fields',
 221:'vine yards',
 223:'olive groves',
 231:'pastures',
 241:'annual with perm. crops',
 242:'complex cultivation patters',
 243:'land principally occupied by agriculture',
 244:'agro forest areas',
 311:'broad leaved forest',
 312:'conferous forest',
 313:'mixed forest',
 321:'natural grassland',
 322:'moors and heathland',
 323: 'scierohllous vegitation',
 324:'transitional woodland shrub',
 331: 'beaches dunes and sand plains',
 332:'bare rock',
 333:'sparsely vegetated areas',
 334:'burnt areas',
 335:'glaciers and perpetual snow',
 411:'inland marshes',
 412:'peat bogs',
 421:'salt marshes',
 422:'salines',
 423:'intertidal flats',
 511:'water courses',
 512:'water bodies',
 521:'costal lagoons',
 522:'estuaries',
 523:'sea and ocean'}
 
 
















In [2]:
landcovers = [key for idx, key in enumerate(clcc) ]

In [3]:
landcovers

[111,
 112,
 121,
 122,
 123,
 124,
 131,
 132,
 133,
 141,
 142,
 211,
 212,
 213,
 221,
 223,
 231,
 241,
 242,
 243,
 244,
 311,
 312,
 313,
 321,
 322,
 323,
 324,
 331,
 332,
 333,
 334,
 335,
 411,
 412,
 421,
 422,
 423,
 511,
 512,
 521,
 522,
 523]

### Algorithm to change a segmentation mask into a feature vector 
#### where each element represnts the number of pixels of each class in each segmentation mask
#### The first part is a dataloader for the segmentation mask and the labels of wild and non wild

In [5]:
#final code
# to do list is to build a transparent model to train on the classes
import glob
import os
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import tifffile as tiff
import time
from tqdm import tqdm


class CustomDataset(Dataset):
    def __init__(self, csv_file, root):
        self.data = pd.read_csv(csv_file)
        
        self.image_paths = sorted(glob.glob(os.path.join(root)+ '/*.*'))
        #self.image_paths = (self.data['file'])
        #print(len(self.image_paths))
        #self.image_paths = sorted(glob.glob(os.path.join('/home/ahmedemam576/working_folder/data/anthroprotect/tiles/lcs', self.image_paths)))
        self.labels = self.data['label']

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        
        image_path = self.image_paths[idx]
        
        label = self.labels[idx]

        
        # we need to open the image as a tiff file and take the first channel representing the corine channel
        image = tiff.imread(image_path)
        image = image.astype('int')
        #print(image.shape)
        image = image[:,:,0]

        return image, label

# Path to the CSV file
csv_file = 'infos.csv'

# Create an instance of the custom dataset
dataset = CustomDataset(csv_file,root = '/home/ahmedemam576/working_folder/data/anthroprotect/tiles/lcs')

# Create a data loader to iterate over the dataset
batch_size = 1
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# code the image to segmentation mask function:
# to extract the corine channel from the path
#print(len(data_loader))
#print(images[0])
# Iterate over the data loader
array = landcovers
feature_array = []
target_labels = []
for images, labels in tqdm(data_loader):
    # Convert images to segmentation mask here using your own logic
    #print('-------loop is working-------')
    segmentation_mask = images
    #print(len(images))
    # Get the unique land cover classes present in the segmentation mask
    land_cover_classes = torch.unique(segmentation_mask)

    # Compute the number of land cover classes and the maximum number of classes expected
    num_classes = len(land_cover_classes)
    max_num_classes = 44  # Set the maximum number of land cover classes expected in the dataset
    
    
    # create a dictionary to have the key the land cover class code and as the ni. of pixels of this specific land cover class as data
    class_counts = {}
    
    
    
    # Generate the vector with the count of pixels for each land cover class
    
    for class_keys in land_cover_classes:
        class_counts = {key: 0 for key in array}

        if class_keys in landcovers: # to make sure that we only have the corine land cover classess
            

        
            class_rep = torch.sum(segmentation_mask == class_keys)
            #class_keys = class_keys.item
            #class_rep = class_rep.item

            class_counts[class_keys.item()] = class_rep.item()
            #print(class_counts)
            #time.sleep(1)
            
    #print('the label is -->',labels.item())
    # Trim the class_counts tensor to the actual number of classes found
    #class_counts = class_counts[:num_classes]
    
    # Print the class counts vector
    '''print("Class Counts:")
    print(class_counts.values())
   
    
    print('feature_array len-->',len(feature_array),'target_labels len-->',len(target_labels))
    print('feature_array -->',(feature_array),'target_labels -->',(target_labels))
    #print(labels)
    #time.sleep(0)'''
    
    
    feature_array.append(list(class_counts.values()))
    target_labels.append(labels.item())


100%|████████████████████████████████████| 23919/23919 [00:46<00:00, 512.34it/s]


### Dataset and dataloader for the scene classification dataset (Anthroprotect dataset)

In [7]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import tifffile as tiff
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.images = self.data['file']
        self.labels = self.data['label']
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_name = self.images[index]  # Assuming the image names are in the first column of the CSV file
        image_label = self.labels[index]
        # Construct the complete image path by joining the folder path and image name
        image_path = os.path.join(self.image_folder, image_name)

        # Open the image using PIL
        image = tiff.imread(image_path)
        
        #choos ethe number of channels you need
        image= image[:,:,:3]
        #####
        
        image = image.astype('uint8')
        #image = image.astype('int')
        # Apply transformations, if provided
        if self.transform:
            #pil_image = Image.fromarray(image)
            image = self.transform(image)
            
        

            

        return image, image_label
    
    
csv_file =  'infos.csv'
image_folder = '~/working_folder/data/anthroprotect/tiles/s2'
transform = transforms.ToTensor()  # Example transformation, you can add more

dataset = CustomDataset(csv_file, image_folder, transform=transform) # Replace with your actual dataset instantiation

# Define the ratio of the dataset to be used for testing
test_ratio = 0.2

# Calculate the number of samples for testing
test_size = int(test_ratio * len(dataset))

# Calculate the number of samples for training
train_size = len(dataset) - test_size

# Split the dataset into training and testing datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader instances for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Example usage





In [8]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),  # Resize the image to (224, 224)
    transforms.ToTensor()  # Convert the PIL Image to tensor
])
scene_dataset= CustomDataset(csv_file, '/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2',transform)


In [9]:
batch_size = 128  # Define the batch size
data_loader = DataLoader(scene_dataset, batch_size=batch_size, shuffle=True)

len(dataset)

23919

### a classifier for scence classification, with a method to extract a the embeddings from the bottelneck

In [11]:
import torch
import torch.nn as nn


import torch
import torch.nn as nn



class DeepClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DeepClassifier, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.embedding = nn.Sequential(
            nn.Linear(256 * 16 * 16, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(10,1),
            nn.Sigmoid())
    
    def forward(self, x):
        features = self.features(x)
        flattened = torch.flatten(features, start_dim=1)
        embedding = self.embedding(flattened)
        output = self.classifier(embedding)
        
        return output, embedding

# Example usage
input_channels = 3
image_size = 256
num_classes = 1
model = DeepClassifier(num_classes)

model = model.to('cuda')

# Training the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


### Training of the scene classifier

In [13]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
# Assuming you have your training data in a PyTorch DataLoader called "train_loader"
for epoch in tqdm(range(num_epochs), desc="Training Progress"):
    # Variables to keep track of accuracy and total samples
    total_loss = 0.0
    total_samples = 0
    
    for data in data_loader:
        inputs, labels = data  
        labels = labels.float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs, encoded_data = model(inputs.float())
        
        
        
        labels = labels.view(-1, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * inputs.size(0)
        
        total_samples += inputs.size(0)
        
    mse = total_loss / total_samples
    print(f"Training MSE: {mse:.4f}")



Training Progress:   5%|█▎                       | 1/20 [01:00<19:03, 60.19s/it]

Training MSE: 0.0792


Training Progress:  10%|██▌                      | 2/20 [02:01<18:11, 60.63s/it]

Training MSE: 0.0658


Training Progress:  15%|███▊                     | 3/20 [03:00<17:04, 60.24s/it]

...

Training MSE: 0.0040


Training Progress:  95%|██████████████████████▊ | 19/20 [19:38<01:03, 63.97s/it]

Training MSE: 0.0027


Training Progress: 100%|████████████████████████| 20/20 [20:42<00:00, 62.12s/it]

Training MSE: 0.0033


### Logistic regressor

In [ ]:
# the transparent model

from sklearn.linear_model import LogisticRegression


from sklearn.preprocessing import StandardScaler

#add the mbedding here

X = feature_array. append(embedding)


##get the target values from the classifier
target = target_labels


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Scale the input features

logreg = LogisticRegression(max_iter=1000)



# Fit the model to the data
logreg.fit(X_scaled, target)

# Print the coefficients of the trained model
coefficients = logreg.coef_[0]
for feature, coefficient in zip(clcc.keys(), coefficients):
    print(f"{feature}: {coefficient}")

### visulaization the coefficients of the LR

In [ ]:
#the coefficients of the trained transparent model
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have a trained logistic regression model named `logreg`
coefficients = coefficients
feature_names = list(clcc.values())  # Assuming `input_data` contains the feature names

# Sort the coefficients and feature names in descending order of absolute magnitude
sorted_indices = np.argsort(np.abs(coefficients))[::-1]
sorted_coefficients = coefficients[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Create the bar plot
plt.figure(figsize=(10, 16))
plt.barh(range(len(sorted_coefficients)), sorted_coefficients, align='center')
plt.yticks(range(len(sorted_coefficients)), sorted_feature_names)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Importance of Coefficients in Logistic Regression Model')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have a trained logistic regression model named `logreg`

  # Assuming `input_data` contains the feature names

# Sort the coefficients and feature names in descending order of absolute magnitude
sorted_indices = np.argsort(np.abs(coefficients))[::-1]
sorted_coefficients = coefficients[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Separate positive and negative coefficients
positive_indices = np.where(sorted_coefficients >= 0)[0]
negative_indices = np.where(sorted_coefficients < 0)[0]

positive_coefficients = sorted_coefficients[positive_indices]
positive_feature_names = [sorted_feature_names[i] for i in positive_indices]

negative_coefficients = sorted_coefficients[negative_indices]
negative_feature_names = [sorted_feature_names[i] for i in negative_indices]

# Create separate plots for positive and negative coefficients using Seaborn
plt.figure(figsize=(12, 6))

# Positive coefficients plot
plt.subplot(1, 2, 1)
sns.barplot(x=positive_coefficients, y=positive_feature_names)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Positive Coefficients')#

# Negative coefficients plot
plt.subplot(1, 2, 2)
sns.barplot(x=negative_coefficients, y=negative_feature_names)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Negative Coefficients')

plt.tight_layout()
plt.show()


### Fine tunning pretrained segmentation model deeplapv3

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import segmentation
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import tifffile as tiff

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file, image_folder, target_folder,transform_image=None,transform_target=None):
        
        self.data = pd.read_csv(csv_file)
        
        
        #the scene input images and the target segmentation maps both have the same name but in different folder
        
        
        
        self.image_paths = self.data['file']
        
        
        self.image_folder = image_folder
        self.target_folder = target_folder
        
        
        self.transform = transform
        self.transform_target = transform_target

    def __getitem__(self, index):
  

        
        # Construct the complete image path by joining the folder path and image name
        image_name = self.image_paths[index] 
        image_path = os.path.join(self.image_folder, image_name)
        print(image_path)
        target_path = os.path.join(self.target_folder, image_name)
        # Open the image using PIL
        image = tiff.imread(image_path)
        target = tiff.imread(target_path)
        
        #choos ethe number of channels you need
        image= image[:,:,:3]
        target.shape
        target= target [:,:,0]
        #####
        print(target.shape)
        
        image = image.astype('uint8')
        target = target.astype('uint8')
        
        

        if self.transform:
            
            image = self.transform(image)
            target = self.transform_target(target)
            
            
            

        return image, target

    def __len__(self):
        return len(self.image_paths)
    #-----------------------------------------------------------------------------------#

# Define the paths to your training data


# Set random seed for reproducibility
torch.manual_seed(42)

# Initialize the model and set it to training mode

model = segmentation.deeplabv3_resnet50(pretrained=True)
model.classifier[-1] = torch.nn.Conv2d(256, 1, kernel_size=1)
model.train()

# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Define the transformation for input images and labels
transform = transforms.Compose([
    transforms.ToPILImage(),
    
    transforms.Resize((512,512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
])
transform_target =transforms.Compose([transforms.ToPILImage(),transforms.Resize((512, 512)),
                                         transforms.ToTensor()])
    #---------------------------------------------------------------------------------------------------#

csv_file =  'infos.csv'
image_folder = '/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2'
target_folder='/home/ahmedemam576/working_folder/data/anthroprotect/tiles/lcs'
    
# Create the custom dataset
dataset = CustomDataset(csv_file, image_folder, target_folder,transform_image=transform, transform_target =transform_target)

# Create the data loader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, targets in dataloader:
        print('input image shape--->',images.shape)
        print('ground truth target shape----->',targets.shape)
        
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        print('models output shape--->',outputs['out'].shape)
        print('target shape--->',targets.shape)
        
        loss = criterion(outputs['out'], targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_model.pth')


/home/ahmedemam576/coding_streak_envionment/Anconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ahmedemam576/coding_streak_envionment/Anconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-03-10_177.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-II_905_2.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])


/home/ahmedemam576/coding_streak_envionment/Anconda3/lib/python3.9/site-packages/torch/autograd/__init__.py:197: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-03-15_113.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_24.81681-64.10019-4_13.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-03-15_43.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-04-8_104.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_18.48087-57.46266_78.tif
(256, 256)
/home/ahmedemam576/wor

models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-03-11_199.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-Ib_174617_51.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_17.28151-62.39233_28.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-02-8_16.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotec

...

models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-03-7_127.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-01-13_173.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-II_824_46.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_16.87133-59.52749-13_222.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprot

models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-Ib_3159-4_79.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-01-20_3.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-04-6_0.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-05-2_152.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthropro

/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_16.65564-56.71377_143.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-03-10_22.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-05-8_1.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-04-3_77.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-Ib_174624_44.tif
(256, 256)
/home/ahmedemam576/working_folder/data/

models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_28.06581-62.07804_37.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-Ib_174624_26.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-II_155143_28.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_13.61147-57.69620-05-5_42.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/wdpa-Ib_

models output shape---> torch.Size([2, 1, 512, 512])
target shape---> torch.Size([2, 1, 512, 512])
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_20.03637-63.83529_40.tif
(256, 256)
/home/ahmedemam576/working_folder/data/anthroprotect/tiles/s2/anthropo_23.70633-61.43592-03-8_86.tif
(256, 256)
input image shape---> torch.Size([2, 3, 512, 512])
ground truth target shape-----> torch.Size([2, 1, 512, 512])
